In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [4]:
dataset_path = 'newoutput1.csv'
df = pd.read_csv(dataset_path)

missing_values = df.isnull().sum()
df.fillna(df.mean(numeric_only=True), inplace=True)

In [5]:
label_encoder = LabelEncoder()
df['Ethnicity'] = label_encoder.fit_transform(df['Ethnicity'])
encoder = OneHotEncoder(sparse=False)
ethnicity_encoded = encoder.fit_transform(df[['Ethnicity']])
ethnicity_encoded_df = pd.DataFrame(ethnicity_encoded, columns=[f'Ethnicity_{i}' for i in range(ethnicity_encoded.shape[1])])
df = pd.concat([df, ethnicity_encoded_df], axis=1)

/Users/siddharthparasher/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
def time_to_minutes(time_str):
    try:
        hours, minutes = map(int, str(time_str).split(':'))
        return hours * 60 + minutes
    except:
        return 0

In [7]:
df['Drink Start Time'] = df['Drink Start Time'].apply(time_to_minutes)

df['BMI'] = (df['Weight (lbs)'] / ((df['Height (ft)'] * 12) + df['Height (in)']) ** 2) * 703

drink_start_time = df['Drink Start Time']

selected_features = df[['nth drink', '# of standard drinks', 'Alcohol Content (g)', 'Time since drink (min)',
                        'Height (ft)', 'Height (in)', 'Weight (lbs)', 'BMI', 'Empty Stomach?', 'Age', 'Sex', 'Ethnicity_0', 'Ethnicity_1', 'Ethnicity_2']]

In [8]:
numeric_features = selected_features.select_dtypes(include=[np.number])
numeric_feature_names = numeric_features.columns.tolist()

scaler = StandardScaler()
numeric_features = scaler.fit_transform(numeric_features)

numeric_features_df = pd.DataFrame(numeric_features, columns=numeric_feature_names)

features = pd.concat([numeric_features_df, drink_start_time], axis=1)

In [9]:
def create_sequences(data, sequence_length):
    sequences = []
    target = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        target.append(data[i+sequence_length])
    return np.array(sequences), np.array(target)

In [10]:
sequence_length = 10
X, y = create_sequences(df['Measured BAC'].values, sequence_length)

In [11]:
train_size = int(0.7 * len(X))
val_size = int(0.2 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

In [12]:

model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))  
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))  
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")

Epoch 1/50
2/2 [==============================] - 4s 599ms/step - loss: 0.8834 - val_loss: 0.0024
Epoch 2/50
2/2 [==============================] - 0s 27ms/step - loss: 0.8805 - val_loss: 0.0020
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 0.8743 - val_loss: 0.0030
Epoch 4/50
2/2 [==============================] - 0s 26ms/step - loss: 0.8740 - val_loss: 0.0051
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.8717 - val_loss: 0.0091
Epoch 6/50
2/2 [==============================] - 0s 25ms/step - loss: 0.8717 - val_loss: 0.0096
Epoch 7/50
2/2 [==============================] - 0s 24ms/step - loss: 0.8707 - val_loss: 0.0096
Epoch 8/50
2/2 [==============================] - 0s 25ms/step - loss: 0.8708 - val_loss: 0.0091
Epoch 9/50
2/2 [==============================] - 0s 24ms/step - loss: 0.8711 - val_loss: 0.0082
Epoch 10/50
2/2 [==============================] - 0s 23ms/step - loss: 0.8691 - val_loss: 0.0084
Epoch 11/50
2/2 [===========